In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
path_to_data = "/content/drive/MyDrive/punct/lenta-ru-news.csv"

In [3]:
with open(path_to_data, "r") as f:
    data = f.read()

In [4]:
data_l = data.split("https://")

In [5]:
def cut_text(text: str):
    d = ""
    try:
        d = text[max(text.index(",\"") + 2, text.index("/,") + 2):text.index("\",")]
    except Exception as e:
        print(e)
    return d

In [ ]:
r = []
for l in data_l:
  d = cut_text(l)
  if d != "":
    r.append(d)

In [10]:
len(data_l)

801143

In [15]:
len(r)

799742

In [16]:
from sklearn.model_selection import train_test_split
tr, te = train_test_split(r,test_size=0.30, shuffle=False)
test, valid = train_test_split(te,test_size=0.33, shuffle=False)

In [17]:
tr[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. \n«Русский инвалид», 16 сентября 1914 года.'

In [28]:
punct_dict = {
    ',': "CM",
    '.': "D",
    '…': "EL",
    '!': "EX",
    '?': "Q",
    ':': "CL"
}
punctuation_symbols = ["\.",",","…","!","\?",":"]
p_s = [x[-1] for x in punctuation_symbols]

In [13]:
import re

In [29]:
def transform_to_save_format(text: str):
    for l in set(re.findall('([A-Z]|[А-Я])', data_l[3])):
        text = re.sub(l, f" {l}", text)
    for s in punctuation_symbols:
        text = re.sub(f"{s}", f"{s[-1]} ", text)
    words = [x for x in text.split(" ") if x != ""]
    r = []
    for w in words:
        if w[-1] in p_s:
            r.append([w[:-1], punct_dict[w[-1]]])
        else:
            r.append([w, "S"])
    return r

In [26]:
text_example="Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. \n«Русский инвалид», 16 сентября 1914 года."

In [ ]:
transform_to_save_format(text_example)

In [20]:
from tqdm import tqdm

In [23]:
def save_texts(collection, path_to):
  with open(path_to, "w") as f:
    for x in tqdm(collection):
      words_pairs = transform_to_save_format(x)
      for word, sym in words_pairs:
        f.write(f"{word}\t{sym}\n")

In [24]:
train_out = "/content/drive/MyDrive/punct/lenta_res_train.tsv"
test_out = "/content/drive/MyDrive/punct/lenta_res_test.tsv"
valid_out = "/content/drive/MyDrive/punct/lenta_res_valid.tsv"

In [31]:
save_texts(tr, train_out)

100%|██████████| 559819/559819 [01:45<00:00, 5294.06it/s]


In [32]:
save_texts(test, test_out)

100%|██████████| 160748/160748 [00:39<00:00, 4116.61it/s]


In [33]:
save_texts(valid, valid_out)

100%|██████████| 79175/79175 [00:21<00:00, 3630.63it/s]


In [36]:
!pip install labml==0.4.125

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.2 MB/s eta 0:00:00


In [42]:
!pip install transformers==4.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 68.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8ed8b735937b781a2631c02c1b9eb825e7abf9f592f6939e90fd9667ed5ab61d
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [3]:
import torch
torch.cuda.is_available()

True

In [ ]:
!cd /content/drive/MyDrive/punct & python /content/drive/MyDrive/punct/train.py \
    --model-name repunct-model \
    --pretrained-model DeepPavlov/rubert-base-cased-sentence \
    --targets S CM D EL EX Q CL \
    --train-data /content/drive/MyDrive/punct/lenta_res_train.tsv \
    --val-data /content/drive/MyDrive/punct/lenta_res_valid.tsv \
    --test-data /content/drive/MyDrive/punct/lenta_res_test.tsv \
    --store-best-weights \
    --epoch 7 \
    --batch-size 4 \
    --augment-rate 0.15 \
    --labml \
    --seed 1 \
    --cuda \
    --save-dir /content/drive/MyDrive/punct/models

Loading model...
Model was loaded.
Loading tokenizer...
Tokenizer was loaded.
Loading train data...
  1% 554383/69586588 [00:46<6:52:45, 2787.48it/s]